In [ ]:
import numpy as np
import pandas as pd
import os
from urllib.parse import urlparse
from urllib.request import HTTPError
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from glob import glob
from pathlib import Path
from scipy.interpolate import interp1d
import pylab as plt
import seaborn as sns
from functools import reduce
from matplotlib import colors
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from pismemulator.utils import load_imbie, load_imbie_csv

secpera = 3.15569259747e7
hist_start = 2008
hist_end = 2014
proj_start = 2015
proj_end = 2100
proj_time = np.arange(proj_start, proj_end + 1)

rcps = [26, 45, 85]
rcpss = [26, 45, 85, "Union"]
rcp_col_dict = {26: "#003466", 45: "#5492CD", 85: "#990002"}
rcp_shade_col_dict = {26: "#4393C3", 45: "#92C5DE", 85: "#F4A582"}
rcp_dict = {26: "RCP 2.6", 45: "RCP 4.5", 85: "RCP 8.5"}


# Load ISMIP6 -- Normalization to year 2008 (hack)

In [ ]:
def load_ismip6_gis(remove_ctrl=True):
    outpath = "."
    v_dir = "v7_CMIP5_pub"
    url = f"https://zenodo.org/record/3939037/files/{v_dir}.zip"

    if remove_ctrl:
        ismip6_filename = "ismip6_gis_ctrl_removed.csv.gz"
    else:
        ismip6_filename = "ismip6_gis_ctrl.csv.gz"

    if os.path.isfile(ismip6_filename):
        df = pd.read_csv(ismip6_filename)
    else:
        print(f"{ismip6_filename} not found locally. Downloading the ISMIP6 archive.")
        if not os.path.isfile(f"{v_dir}.zip"):
            with urlopen(url) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(outpath)
        print("   ...and converting to CSV")
        ismip6_gis_to_csv(v_dir, ismip6_filename, remove_ctrl)
        df = pd.read_csv(ismip6_filename)
    return df


def ismip6_gis_to_csv(basedir, ismip6_filename, remove_ctrl):
    # Now read model output from each of the ISMIP6 files. The information we
    # need is in the file names, not the metadate so this is no fun.
    # Approach is to read each dataset into a dataframe, then concatenate all
    #   dataframes into one Arch dataframe that contains all model runs.
    # Resulting dataframe consists of both historical and projected changes

    ctrl_files = []
    for path in Path(basedir).rglob("*_mm_*_ctrl_proj.nc"):
        ctrl_files.append(path)

    hist_files = []
    for path in Path(basedir).rglob("*_mm_*_historical.nc"):
        hist_files.append(path)

    dfs = []
    for path in Path(basedir).rglob("*_mm_cr_*.nc"):
        # Experiment
        nc = NC(path)
        exp_sle = nc.variables["sle"][:]
        # For comparison with GRACE, we use grounded ice mass, converted to Gt
        exp_mass = nc.variables["limgr"][:] / 1e12
        exp_smb = nc.variables["smb"][:] / 1e12 * secpera

        f = path.name.split(f"scalars_mm_cr_GIS_")[-1].split(".nc")[0].split("_")
        # This is ugly, because of "ITLS_PIK"
        if len(f) == 3:
            group, model, exp = f
        else:
            g1, g2, model, exp = f
            group = f"{g1}_{g2}"

        if exp in ["exp07"]:
            rcp = 26
        else:
            rcp = 85
        # Find the coressponding CTRL Historical simulations
        ctrl_file = [m for m in ctrl_files if (f"_{group}_{model}_" in m.name)][0]
        hist_file = [m for m in hist_files if (f"_{group}_{model}_" in m.name)][0]

        # The last entry of the historical and the first entry of the projection are the same

        # Projection
        nc_ctrl = NC(ctrl_file)
        ctrl_sle = nc_ctrl.variables["sle"][:] - nc_ctrl.variables["sle"][0]
        ctrl_mass = (nc_ctrl.variables["limgr"][:] - nc_ctrl.variables["limgr"][0]) / 1e12
        ctrl_smb = nc_ctrl.variables["smb"][:] / 1e12 * secpera

        # Per email with Heiko on Nov. 13, 2020, stick with just the exp projections alone, without adding back the ctrl projections
        """
        from Heiko:
        "The solution that we chose for ISMIP6 is therefore to remove the ctrl_proj from the projections
        and communicate the numbers as such, i.e. SL contribution for additional forcing after 2014. 
        In our (strong) opinion, the results should never be communicated uncorrected."
        
        Also, point of reference from Goelzer et al., 2020, the ctrl simulations represent mass change
        with the SMB fixed to 1960-1989 levels (no anomaly in SMB) and no change in ice sheet mask.
        So ctrl after the historical spinup represents an abrupt return to an earlier SMB forcing in 2015.
        """

        # Historical
        nc_hist = NC(hist_file)
        hist_sle = nc_hist.variables["sle"][:-1] - nc_hist.variables["sle"][-1]
        hist_mass = (nc_hist.variables["limgr"][:-1] - nc_hist.variables["limgr"][-1]) / 1e12
        hist_smb = nc_hist.variables["smb"][:-1] / 1e12 * secpera
        if remove_ctrl:
            proj_sle = exp_sle
            proj_mass = exp_mass
            proj_smb = exp_smb
        else:
            proj_sle = exp_sle + ctrl_sle
            proj_mass = exp_mass + ctrl_mass
            proj_smb = exp_smb + ctrl_smb

        # Historical simulations start at different years since initialization was left
        # up to the modelers
        hist_time = -np.arange(len(hist_sle))[::-1] + hist_end

        # Let's add the data to the main DataFrame
        m_time = np.hstack((hist_time, proj_time))
        m_sle = -np.hstack((hist_sle, proj_sle)) * 100
        m_sle -= np.interp(2008, m_time, m_sle)
        m_mass = np.hstack((hist_mass, proj_mass))
        m_smb = np.cumsum(np.hstack((hist_smb, proj_smb)))
        m_smb -= np.interp(2088, m_time, m_smb)
        m_d = m_mass - m_smb
        m_mass_rate = np.gradient(np.hstack((hist_mass, proj_mass)))
        m_smb_rate = np.hstack((hist_smb, proj_smb))
        m_d_rate = m_mass_rate - m_smb_rate
        m_mass -= np.interp(2008, m_time, m_mass)

        n = len(m_time)
        dfs.append(
            pd.DataFrame(
                data=np.hstack(
                    [
                        m_time.reshape(-1, 1),
                        m_sle.reshape(-1, 1),
                        m_mass.reshape(-1, 1),
                        m_smb.reshape(-1, 1),
                        m_d.reshape(-1, 1),
                        m_mass_rate.reshape(-1, 1),
                        m_smb_rate.reshape(-1, 1),
                        m_d_rate.reshape(-1, 1),
                        np.repeat(group, n).reshape(-1, 1),
                        np.repeat(model, n).reshape(-1, 1),
                        np.repeat(exp, n).reshape(-1, 1),
                        np.repeat(rcp, n).reshape(-1, 1),
                    ]
                ),
                columns=[
                    "Year",
                    "SLE (cm)",
                    "Cumulative ice sheet mass change (Gt)",
                    "Cumulative surface mass balance anomaly (Gt)",
                    "Cumulative ice dynamics anomaly (Gt)",
                    "Rate of ice sheet mass change (Gt/yr)",
                    "Rate of surface mass balance anomaly (Gt/yr)",
                    "Rate of ice dynamics anomaly (Gt/yr)",
                    "Group",
                    "Model",
                    "Exp",
                    "RCP",
                ],
            )
        )
        # End of working with each model run individually (the path for-loop)

    # Concatenate all DataFrames and convert object types
    df = pd.concat(dfs)
    df = df.astype(
        {
            "Year": float,
            "SLE (cm)": float,
            "Cumulative ice sheet mass change (Gt)": float,
            "Cumulative surface mass balance anomaly (Gt)": float,
            "Cumulative ice dynamics anomaly (Gt)": float,
            "Rate of ice sheet mass change (Gt/yr)": float,
            "Rate of surface mass balance anomaly (Gt/yr)": float,
            "Rate of ice dynamics anomaly (Gt/yr)": float,
            "Model": str,
            "Exp": str,
            "RCP": str,
        }
    )
    df.to_csv(ismip6_filename, compression="gzip")


# Plot function

In [ ]:
def plot_posterior_sle_pdfs(
    out_filename,
    df,
    observed=None,
    rcps=[26, 45, 85],
    ensembles=["AS19", "Flow Calib.", "Flow+Mass Calib."],
    years=[2020, 2100],
    ylim=None,
):

    n_rcps = len(rcps)
    legend_rcp = 85
    alphas = [0.4, 0.7, 1.0]
    m_alphas = alphas[: len(ensembles)]

    fig, axs = plt.subplots(
        n_rcps * 2,
        2,
        sharex="col",
        figsize=[5.8, 4.2],
        gridspec_kw=dict(height_ratios=[0.30 * len(ensembles), 4] * n_rcps),
    )
    fig.subplots_adjust(hspace=0.0, wspace=0)
    for k, rcp in enumerate(rcps):
        for y, year in enumerate(years):
            y_df = df[df["Year"] == year]
            q_df = make_quantile_df(y_df, quantiles=[0.05, 0.16, 0.5, 0.84, 0.95])

            m_df = y_df[y_df["RCP"] == rcp]
            p_df = q_df[q_df["RCP"] == rcp]

            sns.kdeplot(
                data=m_df,
                x="SLE (cm)",
                hue="Ensemble",
                hue_order=ensembles,
                common_norm=False,
                common_grid=True,
                multiple="layer",
                fill=True,
                lw=0,
                palette=[color_tint(rcp_col_dict[rcp], alpha) for alpha in m_alphas],
                ax=axs[k * 2 + 1, y],
            )

            sns.kdeplot(
                data=m_df,
                x="SLE (cm)",
                hue="Ensemble",
                hue_order=ensembles,
                common_norm=False,
                common_grid=True,
                multiple="layer",
                fill=False,
                lw=0.8,
                palette=[color_tint(rcp_col_dict[rcp], alpha) for alpha in m_alphas],
                ax=axs[k * 2 + 1, y],
            )

            for e, ens in enumerate(ensembles):
                s_df = p_df[p_df["Ensemble"] == ens]
                mk_df = y_df[y_df["Ensemble"] == ens]

                alpha = alphas[e]
                m_color = color_tint(rcp_col_dict[rcp], alpha)
                lw = 0.25

                axs[(k * 2), y].vlines(
                    s_df[[0.5]].values[0][0], e, e + 1, colors="k", lw=1
                )

                rect1 = plt.Rectangle(
                    (s_df[[0.05]].values[0][0], e + 0.4),
                    s_df[[0.95]].values[0][0] - s_df[[0.05]].values[0][0],
                    0.2,
                    color=m_color,
                    alpha=1,
                    lw=0,
                )
                rect2 = plt.Rectangle(
                    (s_df[[0.16]].values[0][0], e + 0.2),
                    s_df[[0.84]].values[0][0] - s_df[[0.16]].values[0][0],
                    0.6,
                    color=m_color,
                    alpha=1,
                    lw=0,
                )
                rect3 = plt.Rectangle(
                    (s_df[[0.05]].values[0][0], e + 0.4),
                    s_df[[0.95]].values[0][0] - s_df[[0.05]].values[0][0],
                    0.2,
                    color="k",
                    alpha=1,
                    fill=False,
                    lw=lw,
                )
                rect4 = plt.Rectangle(
                    (s_df[[0.16]].values[0][0], e + 0.2),
                    s_df[[0.84]].values[0][0] - s_df[[0.16]].values[0][0],
                    0.6,
                    color="k",
                    alpha=1,
                    fill=False,
                    lw=lw,
                )

                axs[(k * 2), y].add_patch(rect1)
                axs[(k * 2), y].add_patch(rect3)
                axs[(k * 2), y].add_patch(rect2)
                axs[(k * 2), y].add_patch(rect4)

                axs[(k * 2), y].set_ylabel(None)
                axs[(k * 2), y].axes.xaxis.set_visible(False)
                axs[(k * 2), y].axes.yaxis.set_visible(False)
                sns.despine(ax=axs[(k * 2), y], left=True, bottom=True)
                sns.despine(ax=axs[(k * 2) + 1, y], top=True)

                axs[(k * 2), y].set_ylim(0, len(ensembles))

                if y > 0:
                    axs[k * 2 + 1, y].set_ylabel(None)

                axs[k, y].legend().remove()
                axs[k * 2 + 1, y].legend().remove()

                axs[0, y].set_title(f"Year {year}")
                if ylim is not None:
                    axs[(k * 2) + 1, y].set_ylim(ylim)

                if (k == 0) and (e == 0) and (y == 0):
                    for pctl in [0.05, 0.16, 0.5, 0.84, 0.95]:
                        axs[0, 0].text(
                            s_df[[pctl]].values[0][0],
                            -1.5,
                            int(pctl * 100),
                            ha="center",
                            fontsize=5,
                        )

        if observed is not None:
            obs = observed[
                (observed["Year"] >= years[0]) & (observed["Year"] < years[0] + 1)
            ]
            obs_mean = obs["SLE (cm)"].mean()
            obs_std = obs["SLE uncertainty (cm)"].mean()
            axs[(k * 2) + 1, 0].axvline(obs_mean, c="k", lw=0.5)
            axs[(k * 2) + 1, 0].axvline(
                obs_mean - 2 * obs_std, c="k", lw=0.5, ls="dotted"
            )
            axs[(k * 2) + 1, 0].axvline(
                obs_mean + 2 * obs_std, c="k", lw=0.5, ls="dotted"
            )

    for k, rcp in enumerate(rcps):
        axs[k * 2, 0].text(
            -0.125,
            0.2,
            rcp_dict[rcp],
            transform=axs[k * 2, 0].transAxes,
            fontsize=7,
            fontweight="bold",
            horizontalalignment="left",
        )

    l_as19 = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[0]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Prior (AS19)",
    )
    l_ismip6 = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[0]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Prior (ISMIP6)",
    )
    l_flow = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[1]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Posterior (Flow Calib.)",
    )
    l_mass = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[1]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Posterior (Mass Calib.)",
    )
    l_calib = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[2]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Posterior (Flow+Mass Calib.)",
    )
    l_ismip6_calib = Patch(
        facecolor=color_tint(rcp_col_dict[legend_rcp], alphas[2]),
        edgecolor="0.0",
        linewidth=0.25,
        label="Posterior (ISMIP6 Calib.)",
    )

    ens_label_dict = {
        "AS19": l_as19,
        "Flow Calib.": l_flow,
        "Mass Calib.": l_mass,
        "Flow+Mass Calib.": l_calib,
        "ISMIP6": l_ismip6,
        "ISMIP6 Calib.": l_ismip6_calib,
    }

    legend_1 = axs[-1, 0].legend(
        handles=[ens_label_dict[e] for e in ensembles],
        loc="lower left",
        bbox_to_anchor=(0.4, 0.45, 0, 0),
    )
    legend_1.get_frame().set_linewidth(0.0)
    legend_1.get_frame().set_alpha(0.0)
    axs[-1, 0].add_artist(legend_1)

    if observed is not None:
        l_obs_mean = Line2D(
            [], [], c="k", lw=0.5, ls="solid", label="Observed (IMBIE) mean"
        )
        l_obs_std = Line2D(
            [], [], c="k", lw=0.5, ls="dotted", label="Observed (IMBIE) $\pm2-\sigma$"
        )
        legend_2 = axs[-3, 0].legend(
            handles=[l_obs_mean, l_obs_std],
            loc="lower left",
            bbox_to_anchor=(0.4, 0.45, 0, 0),
        )
        legend_2.get_frame().set_linewidth(0.0)
        legend_2.get_frame().set_alpha(0.0)

    fig.tight_layout()
    fig.savefig(out_filename)
    plt.close(fig)

def color_tint(m_color, alpha):
    m_color = list(colors.to_rgba(m_color))
    m_color[-1] = alpha
    m_color = np.array(m_color) * 255
    return rgba2rgb(m_color) / 255


def rgba2rgb(rgba, background=(255, 255, 255)):

    rgb = np.zeros((3), dtype="float32")
    r, g, b, a = rgba[0], rgba[1], rgba[2], rgba[3]

    a = np.asarray(a, dtype="float32") / 255.0

    R, G, B = background

    rgb[0] = r * a + (1.0 - a) * R
    rgb[1] = g * a + (1.0 - a) * G
    rgb[2] = b * a + (1.0 - a) * B

    return np.asarray(rgb, dtype="uint8")


In [ ]:
def resample_ensemble_by_data(
    observed,
    simulated,
    rcps=[26, 45, 85],
    calibration_start=2010,
    calibration_end=2020,
    fudge_factor=3,
    n_samples=500,
    verbose=False,
    m_var="Mass (Gt)",
    m_var_std="Mass uncertainty (Gt)",
):
    """
    Resampling algorithm by Douglas C. Brinkerhoff


    Parameters
    ----------
    observed : pandas.DataFrame
        A dataframe with observations
    simulated : pandas.DataFrame
        A dataframe with simulations
    calibration_start : float
        Start year for calibration
    calibration_end : float
        End year for calibration
    fudge_factor : float
        Tolerance for simulations. Calculated as fudge_factor * standard deviation of observed
    n_samples : int
        Number of samples to draw.

    """

    observed_calib_time = (observed["Year"] >= calibration_start) & (
        observed["Year"] <= calibration_end
    )
    observed_calib_period = observed[observed_calib_time]
    # print(observed_calib_period)
    # Should we interpolate the simulations at observed time?
    observed_interp_mean = interp1d(
        observed_calib_period["Year"], observed_calib_period[m_var]
    )
    observed_interp_std = interp1d(
        observed_calib_period["Year"], observed_calib_period[m_var_std]
    )

    simulated_calib_time = (simulated["Year"] >= calibration_start) & (
        simulated["Year"] <= calibration_end
    )
    simulated_calib_period = simulated[simulated_calib_time]

    resampled_list = []
    for rcp in rcps:
        log_likes = []
        experiments = np.unique(simulated_calib_period["Experiment"])
        evals = []
        for i in experiments:
            exp_ = simulated_calib_period[
                (simulated_calib_period["Experiment"] == i)
                & (simulated_calib_period["RCP"] == rcp)
            ]
            log_like = 0.0
            for year, exp_mass in zip(exp_["Year"], exp_[m_var]):
                try:
                    observed_mass = observed_interp_mean(year)
                    observed_std = observed_interp_std(year) * fudge_factor
                    log_like -= 0.5 * (
                        (exp_mass - observed_mass) / observed_std
                    ) ** 2 + 0.5 * np.log(2 * np.pi * observed_std**2)
                except ValueError:
                    pass
            if log_like != 0:
                evals.append(i)
                log_likes.append(log_like)
                if verbose:
                    print(f"{rcp_dict[rcp]}, Experiment {i:.0f}: {log_like:.2f}")
        experiments = np.array(evals)
        w = np.array(log_likes)
        w -= w.mean()
        weights = np.exp(w)
        weights /= weights.sum()
        resampled_experiments = np.random.choice(experiments, n_samples, p=weights)
        new_frame = []
        for i in resampled_experiments:
            new_frame.append(
                simulated[(simulated["Experiment"] == i) & (simulated["RCP"] == rcp)]
            )
        simulated_resampled = pd.concat(new_frame)
        resampled_list.append(simulated_resampled)

    simulated_resampled = pd.concat(resampled_list)

    return simulated_resampled



In [ ]:
def make_quantile_df(df, quantiles):
    q_dfs = [
        df.groupby(by=["RCP", "Ensemble"])["SLE (cm)"]
        .quantile(q)
        .reset_index()
        .rename(columns={"SLE (cm)": q})
        for q in quantiles
    ]
    q_df = reduce(lambda df1, df2: pd.merge(df1, df2, on=["RCP", "Ensemble"]), q_dfs)
    a_dfs = [
        df.groupby(by=["Ensemble"])["SLE (cm)"]
        .quantile(q)
        .reset_index()
        .rename(columns={"SLE (cm)": q})
        for q in quantiles
    ]
    a_df = reduce(lambda df1, df2: pd.merge(df1, df2, on=["Ensemble"]), a_dfs)
    a_df["RCP"] = "Union"
    return pd.concat([q_df, a_df]).round(1)


In [ ]:
observed = load_imbie_csv()


In [ ]:
load_ismip6_gis(remove_ctrl=False)
ismip6 = pd.read_csv("ismip6_gis_ctrl.csv.gz")
    


In [ ]:
ismip6["Experiment"] = ismip6["Group"] + ismip6["Model"] + ismip6["Exp"]
ismip6["Mass (Gt)"] = ismip6["Cumulative ice sheet mass change (Gt)"]
ismip6["Ensemble"] = "ISMIP6"
ismip6_calib = resample_ensemble_by_data(observed, ismip6, rcps=[26, 85], calibration_end=2015, fudge_factor=3)
ismip6_calib["Ensemble"] = "ISMIP6 Calib."
ismip6_df = pd.concat([ismip6, ismip6_calib])

In [ ]:
plot_posterior_sle_pdfs("ismip6_calibrated.pdf", ismip6_df.reset_index(), observed, rcps=[26, 85], years=[2015, 2100], ensembles=["ISMIP6", "ISMIP6 Calib."])

In [ ]:
ts_median_palette_dict = {
    "AS19": "0.6",
    "Flow Calib.": "0.3",
    "Mass Calib.": "#e6550d",
    "Flow+Mass Calib.": "0.0",
    "ISMIP6": "0.6",
    "ISMIP6 Calib.": "0.0",
}
palette_dict = {
    "AS19": "#c51b8a",
    "Flow Calib.": "#31a354",
    "Mass Calib.": "#2c7fb8",
    "Flow+Mass Calib.": "0.0",
    "ISMIP6": "#c51b8a",
    "ISMIP6 Calib.": "0.0",
}
ts_fill_palette_dict = {
    "AS19": "0.80",
    "Flow Calib.": "0.70",
    "Mass Calib.": "#fee6ce",
    "Flow+Mass Calib.": "0.60",
    "ISMIP6": "0.80",
    "ISMIP6 Calib.": "0.60",
}

def plot_historical(
    out_filename,
    simulated=None,
    observed=None,
    ensembles=["AS19", "Flow+Mass Calib."],
    quantiles=[0.05, 0.95],
    sigma=2,
    simulated_ctrl=None,
    xlims=[2008, 2021],
    ylims=[-10000, 500],
):
    """
    Plot historical simulations and observations
    """

    fig = plt.figure(num="historical", clear=True, figsize=[4.6, 1.6])
    ax = fig.add_subplot(111)

    if simulated is not None:
        for r, ens in enumerate(ensembles):
            legend_handles = []
            sim = simulated[simulated["Ensemble"] == ens]
            g = sim.groupby(by="Year")["Mass (Gt)"]
            sim_median = g.quantile(0.50)
            sim_low = g.quantile(quantiles[0])
            sim_high = g.quantile(quantiles[-1])

            l_es_median = ax.plot(
                sim_median.index,
                sim_median,
                color=ts_median_palette_dict[ens],
                linewidth=signal_lw,
                label="Median",
            )
            legend_handles.append(l_es_median[0])
            ci = ax.fill_between(
                sim_median.index,
                sim_low,
                sim_high,
                color=ts_fill_palette_dict[ens],
                alpha=0.75,
                linewidth=0.0,
                zorder=-11,
                label=f"{quantiles[0]*100:.0f}-{quantiles[-1]*100:.0f}%",
            )
            legend_handles.append(ci)

            legend = ax.legend(
                handles=legend_handles,
                loc="lower left",
                ncol=1,
                title=ens,
                bbox_to_anchor=(r * 0.2, 0.01),
            )
            legend.get_frame().set_linewidth(0.0)
            legend.get_frame().set_alpha(0.0)
            ax.add_artist(legend)

    if observed is not None:
        legend_handles = []
        obs_line = ax.plot(
            observed["Year"],
            observed["Mass (Gt)"],
            "-",
            color=obs_signal_color,
            linewidth=signal_lw,
            label="Mean",
            zorder=20,
        )
        legend_handles.append(obs_line[0])
        obs_ci = ax.fill_between(
            observed["Year"],
            observed["Mass (Gt)"] - sigma * observed["Mass uncertainty (Gt)"],
            observed["Mass (Gt)"] + sigma * observed["Mass uncertainty (Gt)"],
            color=obs_sigma_color,
            alpha=0.75,
            linewidth=0,
            zorder=5,
            label=f"{sigma}-$\sigma$",
        )
        legend_handles.append(obs_ci)

        if simulated is None:
            r = 0
        legend = ax.legend(
            handles=legend_handles,
            loc="lower left",
            ncol=1,
            title="Observed (IMBIE)",
            bbox_to_anchor=((r + 1.0) * 0.2, 0.01),
        )
        legend.get_frame().set_linewidth(0.0)
        legend.get_frame().set_alpha(0.0)
        ax.add_artist(legend)

    ax.axhline(0, color="k", linestyle="dotted", linewidth=0.6)

    ax.set_xlabel("Year")
    ax.set_ylabel(f"Cumulative mass change\nsince {proj_start} (Gt)")

    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax_sle = ax.twinx()
    ax_sle.set_ylabel(f"Contribution to sea-level \nsince {proj_start} (cm SLE)")
    ax_sle.set_ylim(-np.array(ylims) * gt2cmSLE)

    fig.savefig(out_filename, bbox_inches="tight")
    plt.close(fig)


In [ ]:
signal_lw = 1.0
obs_signal_color = "#6a51a3"
obs_sigma_color = "#cbc9e2"
gt2cmSLE = 1.0 / 362.5 / 10.0

plot_historical("historical-ismip6.pdf", simulated=ismip6_df, observed=observed, ensembles=["ISMIP6", "ISMIP6 Calib."])

In [ ]:
!open *ismip*pdf